In [1]:
import pandas as pd
import os
import numpy as np
os.environ["KERAS_BACKEND"] = "jax" 
import keras

In [3]:
model_dir = "/dtu/p1/johlau/Thesis-Synthex/models/"
models = os.listdir(model_dir)

model_dict = {
    "baseline": [i for i in models if i.startswith("baseline") and i.endswith(".keras")],
    "drr-center": [i for i in models if i.startswith("drr-center") and i.endswith(".keras")],
    "drr-2": [i for i in models if i.startswith("drr-2") and i.endswith(".keras")],
    "drr-5": [i for i in models if i.startswith("drr-5") and i.endswith(".keras")],
    "drr-10": [i for i in models if i.startswith("drr-10") and i.endswith(".keras")],
}

In [4]:
model_dict["drr-10"]

['drr-10_epochs_30_split_0.keras',
 'drr-10_epochs_30_split_1.keras',
 'drr-10_epochs_30_split_2.keras',
 'drr-10_epochs_30_split_3.keras',
 'drr-10_epochs_30_split_4.keras']

In [3]:
data_dir = "/dtu/p1/johlau/Thesis-Synthex/data/"

In [4]:
padchest_test = pd.read_csv(data_dir+ "padchest_dev.csv")
chexpert = pd.read_csv(data_dir+ "chexpert.csv")
chexpert["image_path"] = chexpert["patient_id"] + "-" + chexpert["img_path"].apply(lambda x : x.replace("/", "-"))

chexpert_files = os.listdir("/dtu/p1/johlau/Thesis-Synthex/data/chexpert-cropped")
chexpert_files_df = pd.DataFrame({
    "image_path":chexpert_files
})

chexpert_test = chexpert.merge(chexpert_files_df, how="inner", on="image_path").sample(frac=0.3, random_state=1).reset_index()

chexpert_test = chexpert_test.rename(columns={
    "Pleural Effusion":"Effusion"
})


In [5]:
chexpert_test.to_csv(data_dir + "chexpert_test.csv")

In [7]:
# Function for creating the n-hot encoding
def get_n_hot_encoding(df, labels_to_encode):
    enc = np.zeros((len(df), len(labels_to_encode)))
    for idx, row in df.iterrows():
        for ldx, l in enumerate(labels_to_encode):
            if row[l] == 1:
                enc[idx][ldx] = 1
    return enc

In [8]:
labels_to_encode = ['Effusion', 'Pneumothorax', 'Atelectasis', 'Cardiomegaly', 'Pneumonia']

In [9]:
chexpert_labels = get_n_hot_encoding(chexpert_test, labels_to_encode)

In [ ]:
x_test = np.array([keras.utils.img_to_array(keras.utils.load_img(data_dir + "chexpert-cropped/" + i)) for i in tqdm(chexpert_test["image_path"])])